In [1]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

class NetWork(nn.Module):#nn.Moduleクラスのサブクラス化(=既存のクラスを活用し新しいクラスを作成する)
    def __init__(self,input_feature,output):
        """
        input_feature:データ(特徴)の入力ユニット数
        output:出力結果への出力ユニット数
        """
        super().__init__()#コンストラクタ(=クラスを新しくした瞬間実行される関数)の上書き
        self.fc1 = nn.Linear(input_feature,output,bias=False)#線形変換する
        nn.init.xavier_normal_(self.fc1.weight)#重みをXavierの初期化
        self.fc2 = nn.Softmax(dim=1)#dim=1:行単位でsoftmaxをかけてくれる
    
    def forward(self,x):#フォワードパス(=ニューラルネットワークが初期の入力を層ごとに処理して出力に向けて送る)の定義
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [2]:
x_train = torch.load("X_train.pt")
y_train = torch.load("Y_train.pt")
x_valid = torch.load("X_valid.pt")
y_valid = torch.load("Y_valid.pt")
model = NetWork(300,4)

loss = nn.CrossEntropyLoss(reduction="mean")#クロスエントロピー損失を計算する．meanで損失の平均指定．
dataset = TensorDataset(x_train,y_train)#同じ要素数の2つのtensorを渡し，その値を得る．
dataload = DataLoader(dataset,batch_size=128,shuffle=True)

d_method = optim.SGD(params=model.parameters(),lr=10)
"""
torch.optim.SGD
params:最適化するパラメータの反復可能オブジェクト(for文が回せるやつみたいな)
lr:学習率(https://qiita.com/amateur2020/items/23704993fb191db0020c)
"""

'\ntorch.optim.SGD\nparams:最適化するパラメータの反復可能オブジェクト(for文が回せるやつみたいな)\nlr:学習率(https://qiita.com/amateur2020/items/23704993fb191db0020c)\n'

In [3]:
epoch = 100

#エポック毎にバッチ処理をして損失を計算したりパラメータを更新したりする．
for ep in range (epoch):#epoch回訓練を行う
    for x,y in dataload:#128個(バッチサイズ)のデータxとラベルyを取得
        y_pred = model(x)#モデルの予測
        CE_loss = loss(y_pred,y)#クロスエントロピーの計算
        d_method.zero_grad()#計算を正しく行うために，モデル内のパラメータ勾配を初期化する．
        CE_loss.backward()#誤差逆伝搬の計算(損失を0に近づけるために必要なパラメータの勾配を効率的に計算できる)
        d_method.step()#モデルのパラメータ更新
    if ep % 10 == 0:#epochが10の倍数の時
        torch.save(model.state_dict(), f"knock76_checkpoint_{str(ep)}.pth")
